# Table of Contents
1  Image概述
2  Image operator(mask, clip, select, addBands, export)
3  ImageCollection概述
4  ImageCollection operator(filter, select, set/get, calculation, toList)
5  Example 1. Calculate NDVI for Jiangxi and Extract NDWI for poyang Lake between 1982 to 2021;
6  Example 2. Download huge ImageCollection from google earth engine; -> Download Images
7  Example 3. Extract NDVI values of climate stations from 1982 to 2021; -> Spatial analysis
8  Example 4. Supervised Classification -> Images Classification
9  Example 5. Unsupervised Classification (clustering) 

In [1]:
import ee, os
import geemap
ee.Initialize()
Map = geemap.Map()
# Map = geemap.Map(center=(39, 110), zoom=4)
# Map

In [3]:
# -------------------------------------------------------
# 2  ImageCollection 去云
#  Calculate NDVI for Jiangxi
# -------------------------------------------------------
year = 2019, 2021
doy = '-01-01', '-12-31'

start_date = str(year[0]) + doy[0]
end_date = str(year[1]) + doy[1]

# jx_bound = ee.FeatureCollection('users/yehuigeo/jx_bound')
jx_bound = geemap.shp_to_ee(r'D:\work\new\shp\fw.shp').geometry()
# Map.addLayer(jx_bound,{},'fw')


In [4]:
def cloudMaskL457(image):
    qaMask = image.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).eq(0)
    saturationMask = image.select('QA_RADSAT').eq(0)
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBand = image.select('ST_B6').multiply(0.00341802).add(149.0)

    return image.addBands(opticalBands, None, True)\
                 .addBands(thermalBand, None, True)\
                 .updateMask(qaMask).updateMask(saturationMask)\
                 .copyProperties(image, ["system:time_start",'system:id'])

def maskS2clouds(image):   # This function was used to mask the clouds for sentinel-2
    qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
             qa.bitwiseAnd(cirrusBitMask).eq(0))

  # Return the masked and scaled data, without the QA bands.
    return image.updateMask(mask).divide(10000).select("B.*").copyProperties(image, ["system:time_start",'system:id'])

def maskL8sr(image):
    qaMask = image.select('QA_PIXEL').bitwiseAnd(int('11111',2)).eq(0);
    saturationMask = image.select('QA_RADSAT').eq(0);

#   // Apply the scaling factors to the appropriate bands.
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2);
    thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0);
    
    return image.addBands(opticalBands, None, True)\
                 .addBands(thermalBands, None, True)\
                 .updateMask(qaMask)\
                 .updateMask(saturationMask)\
                 .copyProperties(image, ["system:time_start",'system:id'])

In [6]:
# Landsat 3 MSS Collection 1 Tier 1 Raw Scenes
# Dataset Availability
# 1978-03-05 - 1983-03-31
# Landsat 5+7 scenes
# Dataset Availability
# 1984 - 2012
# get landsat 5
# collection5 = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2') \
#     .filterBounds(jx_bound) \
#     .filterDate(start_date, end_date) \
#     .map(cloudMaskL457) \
#     .select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7'])
# print('Landsat5')
# print(collection5.size().getInfo())
# Landsat 7 Collection 1 Tier 1 Raw Scenes
# Dataset Availability
# 1999 - 2021
# get landsat 7
# https://developers.google.com/earth-engine/datasets/catalog/landsat
# https://www.usgs.gov/landsat-missions/landsat-collection-2
bands = ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7']
collection8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')\
            .filterBounds(jx_bound) \
            .filterDate(start_date, end_date) \
            .map(maskL8sr)\
            .select(bands)
# print(collection8.size().getInfo())

In [7]:
def glst1(pix):
    tp = pix.split('_')[0]
    pr = pix.split('_')[1]
    yr = pix.split('_')[-1][:4]
    mn = pix.split('_')[-1][4:6]
    dy = pix.split('_')[-1][-2::] 
    return list([tp,pr,yr,mn,dy])
# 通过数据id属性来split获取时间等信息
def glst2(pix):
    tp = pix.split('_')[1]
    pr = pix.split('_')[2]
    yr = pix.split('_')[-1][:4]
    mn = pix.split('_')[-1][4:6]
    dy = pix.split('_')[-1][-2::] 
    return list([tp,pr,yr,mn,dy])

In [11]:
col57_pix = collection8.aggregate_array('system:index').getInfo()
col57_pix_lst = list(map(glst2, col57_pix))

ProxyError: ('Cannot connect to proxy.', ConnectionResetError(10054, '远程主机强迫关闭了一个现有的连接。', None, 10054, None))

In [10]:
import pandas as pd
ln = ['source','path_row','year','month','day']
df57 = pd.DataFrame(col57_pix_lst, columns = ln)
# 输出数据的前10行
df57.head(10)

NameError: name 'col57_pix_lst' is not defined

In [8]:
df57.to_csv(r'C:\Users\xiong\Downloads\temp\pe.txt')

In [9]:
image = collection8.filter(ee.Filter.calendarRange(1, 1, 'month')).mosaic()

In [10]:
Map.addLayer(image,{},"image")
Map

KeyboardInterrupt: 

In [ ]:
# from GEE_xiong import ee_export

# ee_export(jx_bound,image,r'C:\Users\xiong\Downloads\data\df',100)

In [ ]:
a

df_mn = df57.groupby(['year','month']).size().reset_index(name='counts')
# df_mn = df.groupby(['year','month']).agg(images=('day','count')).reset_index()
df_mn[['year','month']] = df_mn[['year','month']].apply(pd.to_numeric)
# df_mn['year'] = pd.to_numeric(df_mn.year, errors='coerce')
# df_gp.dtypes
df_mn.head(10)
# -------------------------------


In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# df_pixs = df_gp[['month','year','count']]
df_pixs = df_mn.pivot('month','year','counts')
# 将无数据的位置的值设为0
df_pixs = df_pixs.fillna(0)

plt.figure(figsize=(14,6))
ax = sns.heatmap(df_pixs, annot=True)
# clmap = sns.cubehelix_palette(start=.5, rot=-.75, as_cmap=True)
# ax = sns.heatmap(df_pixs, annot=True, cmap=clmap)


In [ ]:
df_id = df57.groupby(['year','path_row']).size().reset_index(name='counts')
df_id[['year','path_row']] = df_id[['year','path_row']].apply(pd.to_numeric)
df_id.head(10)

In [ ]:
df_px = df_id.pivot('path_row','year','counts')
df_px = df_px.fillna(0)

plt.figure(figsize=(14,6))
clmap = sns.cubehelix_palette(start=.5, rot=-.75, as_cmap=True)
ax = sns.heatmap(df_px, annot=True, cmap=clmap)